In [1]:
import pandas as pd
import numpy as np
import nltk 
import matplotlib.pyplot as plt

from gensim.models import Word2Vec

In [3]:
import json

# Define a function to load data from a file
def load_data(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Load the data from the three files
test_data_path = 'test_bio_labels.json'
train_data_path = 'train_bio_labels.json'
val_data_path = 'val_bio_labels.json'

test_data = load_data(test_data_path)
train_data = load_data(train_data_path)
val_data = load_data(val_data_path)
# Adjusting approach to properly display the data structure and a sample

# Function to safely extract samples from data which might be a dictionary or list
def get_sample(data):
    if isinstance(data, dict):
        # If the data is a dictionary, return a part of it
        return {k: data[k] for k in list(data)[:2]}
    elif isinstance(data, list):
        # If the data is a list, return the first two elements
        return data[:2]
    else:
        return "Unsupported data type"

# Get samples from each dataset
test_data_sample = get_sample(test_data)
train_data_sample = get_sample(train_data)
val_data_sample = get_sample(val_data)

{
    "test_data_sample": test_data_sample,
    "train_data_sample": train_data_sample,
    "val_data_sample": val_data_sample,
}

{'test_data_sample': {'03f3901e95ed493b866bd7807f623bc0': {'text': "True, our Constitution has no 'due process' clause or the VIII Amendment; but, in this branch of law, after R.C. Cooper v. Union of India, (1970) 1 SCC 248 and Maneka Gandhi v. Union of India, (1978) 1 SCC 248, the consequence is the same.",
   'labels': ['O',
    'O',
    'B_STATUTE',
    'I_STATUTE',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'O',
    'B_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'B_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'I_PRECEDENT',
    'O',
    'O',
    'O',
    'O',
    'O']},
  'b0311cba3aac4d909eec6e156c059617': {'text': 

In [4]:
import pandas as pd
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import TweetTokenizer
import nltk

# Convert dictionary data to DataFrame
train_df = pd.DataFrame.from_dict(train_data, orient='index').reset_index(drop=True)
test_df = pd.DataFrame.from_dict(test_data, orient='index').reset_index(drop=True)
val_df = pd.DataFrame.from_dict(val_data, orient='index').reset_index(drop=True)

# Display a sample from the train DataFrame to check preprocessing
train_df.head()

,text,labels
0,"Therefore, while interpreting statutory provis...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,The petitioner in W.P.No.15821 of 2008 was nev...,"[O, O, O, B_CASE_NUMBER, I_CASE_NUMBER, I_CASE..."
2,"The factum of accident, allegation of rash and...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B_OTHE..."
3,..36.. \n\n W.A.No.655/2012 & others Meaning t...,"[O, O, B_CASE_NUMBER, I_CASE_NUMBER, O, O, O, ..."
4,The law on this issue is well settled and the ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [6]:
# fasttext model
from gensim.models import FastText
from gensim.models import Word2Vec

# Import the FastText model
from gensim.models import FastText

# Define the FastText model and train it on all the sentences from the three datasets
train_sentences = train_df['text'].apply(lambda x: x.split()).tolist()
test_sentences = test_df['text'].apply(lambda x: x.split()).tolist()
val_sentences = val_df['text'].apply(lambda x: x.split()).tolist()

#combine all sentences
all_sentences = train_sentences + test_sentences + val_sentences

# Train the FastText model
fasttext_model = FastText(sentences=all_sentences, vector_size=512, window=5, min_count=5, sg=1)

#example
example_sentence = train_sentences[0]

# Get the word vectors for the words in the example sentence
word_vectors = [fasttext_model.wv[word] for word in example_sentence if word in fasttext_model.wv.key_to_index]

# now similarly, convert all the sentences to vectors
train_vectors = [fasttext_model.wv[word] for sentence in train_sentences for word in sentence if word in fasttext_model.wv.key_to_index]
test_vectors = [fasttext_model.wv[word] for sentence in test_sentences for word in sentence if word in fasttext_model.wv.key_to_index]
val_vectors = [fasttext_model.wv[word] for sentence in val_sentences for word in sentence if word in fasttext_model.wv.key_to_index]
